In [1]:
import numpy as np
import supervision as sv
import cv2
from ultralytics import YOLO

In [7]:
model = YOLO(r"C:\Users\dolap\OneDrive\Documents\DOLAPO\data-analysis\fish-behavior-analysis\model\best.pt")

In [ ]:
tracker = sv.ByteTrack()
mask_annotator = sv.MaskAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

In [ ]:
def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {model.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = mask_annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

In [ ]:
sv.process_video(
    source_path= r"C:\Users\dolap\OneDrive\Documents\DOLAPO\data-analysis\fish-behavior-analysis\data\test_video.mp4",
    target_path="result.mp4",
    callback=callback
)

print("Processing complete. The result is saved as 'result.mp4'.")


0: 480x800 1 fish, 1145.9ms
Speed: 103.0ms preprocess, 1145.9ms inference, 43.9ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 215.5ms
Speed: 4.0ms preprocess, 215.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 195.7ms
Speed: 5.0ms preprocess, 195.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 196.6ms
Speed: 4.2ms preprocess, 196.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 186.1ms
Speed: 5.7ms preprocess, 186.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 201.1ms
Speed: 5.6ms preprocess, 201.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 188.9ms
Speed: 5.5ms preprocess, 188.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fish, 182.6ms
Speed: 6.4ms preprocess, 182.6ms inference, 1.6ms postprocess per image at shape (1, 